In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = ["ERCOT"]

In [5]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

13it [00:01,  7.69it/s]


In [6]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

ERCOT


In [7]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [8]:
for iso in ISOS:
    clean_final(iso, "all_data")

                               hour  regdn  regup    rrs  nspin  \
time_tomerge                                                      
2014-01-01 06:00:00+00:00      1:00   5.65   9.72  11.72   1.00   
2014-01-01 07:00:00+00:00      2:00   5.55   8.22   8.73   1.00   
2014-01-01 08:00:00+00:00      3:00   5.55   7.57   8.11   1.00   
2014-01-01 09:00:00+00:00      4:00   8.11   4.81   7.25   1.00   
2014-01-01 10:00:00+00:00      5:00   8.11  10.00  10.00   1.00   
...                             ...    ...    ...    ...    ...   
2019-01-01 01:00:00+00:00     20:00   4.01   4.00   9.24   0.89   
2019-01-01 02:00:00+00:00     21:00   3.50   4.00   7.21   1.28   
2019-01-01 03:00:00+00:00     22:00   4.01   3.22   6.02   0.94   
2019-01-01 04:00:00+00:00     23:00   7.15   6.18   6.38   0.97   
2019-01-01 05:00:00+00:00  24:00:00   4.58   4.00   4.61   0.97   

                           quantity_nsrs  quantity_regdn  quantity_regup  \
time_tomerge                                        

In [9]:
ISO_final["ERCOT"].columns

Index(['hour', 'regdn', 'regup', 'rrs', 'nspin', 'quantity_nsrs',
       'quantity_regdn', 'quantity_regup', 'quantity_rrs', 'congestion_da',
       'congestion_rt', 'energy_da', 'energy_rt', 'unnamed: 0', 'gas_price',
       'gen_biomass', 'gen_coal', 'gen_gas', 'gen_gascc', 'gen_hydro',
       'gen_nuclear', 'gen_other', 'gen_solar', 'gen_wind', 'load'],
      dtype='object')

#FOR COMBING DATA
ISO_done = ["ERCOT"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ERCOT/new"
for file in (listdir_nohidden(FILE_PATH)):
    if file in ["ancillary_price", "energy_da", "energy_rt", "generation", "load"]:
        dfs = []
        for f in tqdm(listdir_nohidden(f"{FILE_PATH}/{file}")):
            df = pd.read_csv(f"{FILE_PATH}/{file}/{f}")
            #df['Date'] = [f[14:24].replace("_", "-") for _ in range(len(df))]
            dfs.append(df)

        temp = pd.concat(dfs)
        os.makedirs(f"{FILE_PATH}/combined", exist_ok=True)
        temp.to_csv(f"{FILE_PATH}/combined/{file}.csv")

In [10]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ERCOT/new"
ancillary = pd.read_csv(f"{FILE_PATH}/combined/ancillary_price.csv")
ancillary = ancillary[ancillary["Repeated Hour Flag"] == "N"].reset_index(drop=True)

print(ancillary)

       Unnamed: 0 Delivery Date Hour Ending Repeated Hour Flag  REGDN  REGUP   \
0               0    01/01/2019       01:00                  N   5.26    5.00   
1               1    01/01/2019       02:00                  N   3.98    4.37   
2               2    01/01/2019       03:00                  N   3.50    4.44   
3               3    01/01/2019       04:00                  N   3.47    5.00   
4               4    01/01/2019       05:00                  N   8.00    7.00   
...           ...           ...         ...                ...    ...     ...   
35055        8779    12/31/2020       20:00                  N   0.68    1.00   
35056        8780    12/31/2020       21:00                  N   0.66    1.00   
35057        8781    12/31/2020       22:00                  N   3.50    3.80   
35058        8782    12/31/2020       23:00                  N   6.27    4.21   
35059        8783    12/31/2020       24:00                  N   6.59    3.80   

         RRS  NSPIN  
0    

In [11]:
times_tomerge = []
for i in tqdm(range(len(ancillary))):
    times_tomerge.append(datetime.strptime(str(ancillary["Delivery Date"][i]), "%m/%d/%Y") + timedelta(hours = int(ancillary["Hour Ending"][i][:2])))

100%|██████████| 35060/35060 [00:01<00:00, 33193.44it/s]


In [12]:
ancillary["time_tomerge"] = times_tomerge
ancillary = ancillary.sort_values(by="time_tomerge")

##RENAME

In [13]:
def clean_energy_da_df(energy: pd.DataFrame):
    energy = energy.iloc[[True if energy["Settlement Point"][i][:2] == "HB" else False for i in range(len(energy))],:]
    energy = energy[energy["Repeated Hour Flag"] == "N"].reset_index(drop=True)
    energy["Settlement Point Price"] = energy["Settlement Point Price"].astype(float)
    energy = pd.DataFrame(energy.groupby(["Delivery Date", "Hour Ending"])["Settlement Point Price"].mean()).rename({"value": "energy_da"})
    time_tomerge = []
    for i in tqdm(range(len(energy))):
        time_tomerge.append(datetime.strptime(str(energy.index[i][0]), "%m/%d/%Y") + timedelta(hours = int(energy.index[i][1][:2])))
    energy["time_tomerge"] = time_tomerge
    energy = energy.reset_index(drop=True)
    return energy

In [34]:
energy_da = pd.read_csv(f"{FILE_PATH}/combined/energy_da.csv", thousands = ",")

In [15]:
clean_energy_da = clean_energy_da_df(energy_da).rename(columns={"Settlement Point Price": "energy_da"})
clean_energy_da

100%|██████████| 35060/35060 [00:01<00:00, 25076.57it/s]


,energy_da,time_tomerge
0,20.533333,2019-01-01 01:00:00
1,19.813333,2019-01-01 02:00:00
2,19.593333,2019-01-01 03:00:00
3,19.241667,2019-01-01 04:00:00
4,21.266667,2019-01-01 05:00:00
...,...,...
35055,13.160000,2022-12-31 20:00:00
35056,11.960000,2022-12-31 21:00:00
35057,11.162857,2022-12-31 22:00:00
35058,8.515714,2022-12-31 23:00:00


In [16]:
def clean_energy_rt_df(energy: pd.DataFrame):
    energy = energy.iloc[[True if energy["Settlement Point Type"][i] == "HU" else False for i in range(len(energy))],:]
    energy = energy[energy["Repeated Hour Flag"] == "N"].reset_index(drop=True)
    energy["Settlement Point Price"] = energy["Settlement Point Price"].astype(float)
    energy = pd.DataFrame(energy.groupby(["Delivery Date", "Delivery Hour"])["Settlement Point Price"].mean()).rename({"value": "energy_rt"})
    time_tomerge = []
    for i in tqdm(range(len(energy))):
        time_tomerge.append(datetime.strptime(str(energy.index[i][0]), "%m/%d/%Y") + timedelta(hours = int(energy.index[i][1])))
    energy["time_tomerge"] = time_tomerge
    energy = energy.reset_index(drop=True)
    return energy

In [17]:
energy_rt = pd.read_csv(f"{FILE_PATH}/combined/energy_rt.csv", thousands = ",")
clean_energy_rt = clean_energy_rt_df(energy_rt)
clean_energy_rt

100%|██████████| 35060/35060 [00:01<00:00, 27062.79it/s]


,Settlement Point Price,time_tomerge
0,13.808125,2019-01-01 01:00:00
1,14.866875,2019-01-01 02:00:00
2,15.502500,2019-01-01 03:00:00
3,16.103125,2019-01-01 04:00:00
4,16.032500,2019-01-01 05:00:00
...,...,...
35055,9.171500,2022-12-31 20:00:00
35056,7.474500,2022-12-31 21:00:00
35057,2.442000,2022-12-31 22:00:00
35058,-1.639500,2022-12-31 23:00:00


In [18]:
np.unique(energy_rt["Settlement Point Type"])

array(['8:00:39 AM', '8:03:42 AM', '8:03:53 AM', 'AH', 'HU', 'LZ', 'LZEW',
       'SH'], dtype=object)

In [19]:
load = pd.read_csv(f"{FILE_PATH}/combined/load.csv")
load

,Unnamed: 0,HourEnding,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT,Hour Ending
0,0,01/01/2020 01:00,"9,890.09","1,364.28","3,721.97",773.88,"11,719.86","2,707.84","5,473.25","1,172.94","36,824.12",NaN
1,1,01/01/2020 02:00,"9,751.59","1,369.57","3,735.51",775.06,"11,639.46","2,706.84","5,387.38","1,165.95","36,531.37",NaN
2,2,01/01/2020 03:00,"9,603.42","1,373.53","3,735.85",774.15,"11,515.42","2,650.38","5,270.33","1,149.08","36,072.17",NaN
3,3,01/01/2020 04:00,"9,501.59","1,365.63","3,743.52",778.10,"11,518.36","2,586.71","5,200.96","1,141.30","35,836.16",NaN
4,4,01/01/2020 05:00,"9,499.43","1,385.60","3,753.90",784.08,"11,649.29","2,548.85","5,230.30","1,147.09","35,998.54",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,8755,12/31/2019 20:00,"11,109.26","1,417.31","3,637.46",825.76,"13,164.66","3,181.36","6,453.70","1,290.26","41,079.76",NaN
35060,8756,12/31/2019 21:00,"10,846.50","1,411.76","3,639.02",822.19,"12,928.91","3,044.65","6,175.76","1,276.75","40,145.54",NaN
35061,8757,12/31/2019 22:00,"10,592.54","1,403.99","3,639.46",819.39,"12,716.27","2,930.56","5,968.29","1,260.09","39,330.60",NaN
35062,8758,12/31/2019 23:00,"10,304.62","1,387.05","3,630.18",804.71,"12,413.09","2,829.53","5,769.75","1,238.41","38,377.36",NaN


In [20]:
time_tomerge = []

for i in tqdm(range(len(load))):
    try:
        time_tomerge.append(datetime.strptime(str(load["HourEnding"][i]), "%m/%d/%Y %H:%M")-timedelta(hours=1))
    except:
        time_tomerge.append(time_tomerge[i-1]+timedelta(hours=1))

100%|██████████| 35064/35064 [00:00<00:00, 51329.45it/s]


In [21]:
load["time_tomerge"] = time_tomerge

In [22]:
generation = pd.read_csv(f"{FILE_PATH}/combined/generation.csv", thousands = ",")


In [23]:
print(generation)

       Unnamed: 0        Date     Fuel Settlement Type     Total    0:15  \
0               0  10/01/2019  Biomass           FINAL     842.8     8.8   
1               1  10/01/2019     Coal           FINAL  230576.6  2423.3   
2               2  10/01/2019      Gas           FINAL  127543.0   328.4   
3               3  10/01/2019   Gas-CC           FINAL  441313.6  3973.8   
4               4  10/01/2019    Hydro           FINAL     936.2     2.2   
...           ...         ...      ...             ...       ...     ...   
13509         265  09/30/2019    Hydro           FINAL     967.8     4.3   
13510         266  09/30/2019  Nuclear           FINAL  119458.2  1244.2   
13511         267  09/30/2019    Other           FINAL      38.4     1.8   
13512         268  09/30/2019    Solar           FINAL    6180.5     0.0   
13513         269  09/30/2019     Wind           FINAL  351701.0  3812.0   

         0:30    0:45    1:00    1:15  ...   22:45   23:00   23:15   23:30  \
0        

In [24]:
test = generation.pivot(index=generation.columns[5:], columns='Fuel', values = "Date")

test.columns

Index(['Biomass', 'Coal', 'Gas', 'Gas-CC', 'Hydro', 'Nuclear', 'Other',
       'Solar', 'WSL', 'Wind'],
      dtype='object', name='Fuel')

In [25]:
generation2 = pd.DataFrame()
for col in generation.columns[5:]:
    temp = generation.pivot(index="Date", columns = "Fuel", values = col)
    if "DST" not in col:
        temp["Hour"] = [int(col.split(":")[0]) for _ in range(len(temp)) if "DST" not in col]
        generation2 = pd.concat([generation2, temp])
    print(generation2)

Fuel        Biomass    Coal    Gas  Gas-CC  Hydro  Nuclear  Other  Solar  \
Date                                                                       
01/01/2019     12.3  1529.1  180.6  2947.6   46.4    974.0    0.6    0.0   
01/01/2020      8.9  1424.6  158.9  3173.1    6.4   1274.8   -0.2    0.0   
01/01/2021      7.4  1562.7  322.2  4320.1    9.0   1275.7    0.3    0.0   
01/01/2022      5.6  2762.6  302.9  2199.8    8.0   1269.9    5.9    0.0   
01/02/2019     12.0  2754.8  519.7  4944.6   53.0    972.9    0.2    0.0   
...             ...     ...    ...     ...    ...      ...    ...    ...   
12/30/2022      3.4  1919.4  212.4  4399.1    4.8   1277.9   26.7    0.1   
12/31/2019      8.7  2161.9  244.8  5542.2    3.7   1274.6    0.1    0.0   
12/31/2020      7.4  1506.3  221.4  2920.3    7.4   1277.1    0.0    0.0   
12/31/2021      5.6  1753.1  127.0  1950.8    7.7   1270.4  -16.9    0.0   
12/31/2022      2.7  1489.5  195.8  2462.6    4.0   1278.6    2.6    0.0   

Fuel       

In [26]:
time_tomerge = [datetime.strptime(str(generation2.index[i]), "%m/%d/%Y")+timedelta(hours=float(generation2["Hour"][i])) for i in range(len(generation2))]


In [27]:
generation2["time_tomerge"] = time_tomerge
generation2 = generation2.drop(columns=["Hour"]).sort_values(by="time_tomerge").reset_index(drop=True).rename(columns={"Biomass": "gen_biomas", "Coal": "gen_coal", "Gas": "gen_gas", "Gas-CC": "gen_gascc", "Hydro": "gen_hydro", "Nuclear": "gen_nuclear", "Other": "gen_other", "Solar": "gen_solar", "WSL": "gen_wsl", "Wind": "gen_wind"})
print(generation2)

Fuel    gen_biomas  gen_coal  gen_gas  gen_gascc  gen_hydro  gen_nuclear  \
0             12.3    1529.1    180.6     2947.6       46.4        974.0   
1             12.0    2858.1    525.1     4977.0       40.1        972.8   
2             12.3    1535.7    146.8     3014.7       46.9        974.0   
3             12.2    1520.3    160.3     2948.5       46.9        973.9   
4             12.3    1587.9    145.8     2976.2       46.7        973.6   
...            ...       ...      ...        ...        ...          ...   
140251         2.8    1117.3    169.4     1492.9        1.1       1277.6   
140252         2.7    1031.2    164.4     1446.7        1.1       1277.5   
140253         2.8     988.1    164.0     1442.4        1.1       1277.5   
140254         2.7    1047.0    166.1     1447.5        1.1       1277.6   
140255         2.7    1014.4    163.6     1440.7        1.1       1277.3   

Fuel    gen_other  gen_solar  gen_wsl  gen_wind        time_tomerge  
0             0.6

In [28]:
generation2 = generation2.groupby(["time_tomerge"]).mean().reset_index()

In [29]:
ISO_final["ERCOT"].columns

Index(['hour', 'regdn', 'regup', 'rrs', 'nspin', 'quantity_nsrs',
       'quantity_regdn', 'quantity_regup', 'quantity_rrs', 'congestion_da',
       'congestion_rt', 'energy_da', 'energy_rt', 'unnamed: 0', 'gas_price',
       'gen_biomass', 'gen_coal', 'gen_gas', 'gen_gascc', 'gen_hydro',
       'gen_nuclear', 'gen_other', 'gen_solar', 'gen_wind', 'load'],
      dtype='object')

In [30]:
ancillary = ancillary.drop(columns = ["Unnamed: 0", "Delivery Date", "Hour Ending", "Repeated Hour Flag"]).rename(columns = {"REGDN": "regdn", "REGUP ": "regup", "RRS": "rrs", "NSPIN": "nspin"})
ancillary.columns

Index(['regdn', 'regup', 'rrs', 'nspin', 'time_tomerge'], dtype='object')

In [31]:
load = load.drop(columns=["Unnamed: 0", "HourEnding", "COAST", "EAST", "FWEST", "NORTH", "SOUTH", "NCENT", "SCENT", "WEST", "Hour Ending"]).rename(columns = {"ERCOT": "load"})

In [32]:
clean_energy_rt = clean_energy_rt.rename(columns={"Settlement Point Price": "energy_rt"})

In [33]:
dfs = [ancillary, clean_energy_da, clean_energy_rt, load, generation2]
combined = ancillary
for df in dfs[1:]:
    combined = combined.merge(df, on = "time_tomerge")

In [210]:
combined

,regdn,regup,rrs,nspin,time_tomerge,energy_da,energy_rt,load,gen_biomas,gen_coal,gen_gas,gen_gascc,gen_hydro,gen_nuclear,gen_other,gen_solar,gen_wsl,gen_wind
0,5.26,5.00,6.04,1.08,2019-01-01 01:00:00,20.533333,13.808125,"37,258.99",12.225,1593.075,145.750,2983.325,46.700,973.675,0.575,0.00,NaN,3578.450
1,3.98,4.37,5.90,1.05,2019-01-01 02:00:00,19.813333,14.866875,"37,300.19",12.300,1571.375,147.375,3063.625,46.725,973.700,0.275,0.00,NaN,3485.250
2,3.50,4.44,5.01,0.93,2019-01-01 03:00:00,19.593333,15.502500,"37,423.54",12.250,1615.050,149.175,3155.725,46.825,973.550,0.850,0.00,NaN,3381.975
3,3.47,5.00,5.63,0.93,2019-01-01 04:00:00,19.241667,16.103125,"37,895.21",12.100,1639.525,196.925,3229.850,46.700,973.400,0.400,0.00,NaN,3343.475
4,8.00,7.00,7.02,0.94,2019-01-01 05:00:00,21.266667,16.032500,"38,825.09",12.050,1708.550,277.750,3462.950,53.500,973.550,0.875,0.00,NaN,3158.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35054,7.00,2.29,1.95,1.00,2022-12-31 19:00:00,16.875714,17.927000,"40,881.88",2.725,1257.600,225.675,2501.300,1.175,1277.500,19.675,0.00,-12.775,5041.775
35055,3.50,1.50,1.65,1.00,2022-12-31 20:00:00,13.160000,9.171500,"39,635.70",2.700,1177.875,219.925,2119.450,1.100,1277.700,12.050,0.05,-25.325,5334.850
35056,4.51,1.50,1.50,0.99,2022-12-31 21:00:00,11.960000,7.474500,"38,605.05",2.725,1174.925,218.600,1807.950,1.150,1277.675,6.175,0.00,-22.725,5489.775
35057,5.46,2.45,1.65,0.99,2022-12-31 22:00:00,11.162857,2.442000,"37,400.30",2.750,1135.575,183.875,1523.650,1.125,1277.425,8.525,0.00,-32.600,5577.700


In [205]:
old = ISO_final["ERCOT"].drop(columns = ["hour", "unnamed: 0"])
old.index = old.index-timedelta(hours=6)

In [211]:
combined["time_tomerge"] = pd.to_datetime(combined["time_tomerge"], utc=True)
combined = combined.set_index("time_tomerge")
combined

,regdn,regup,rrs,nspin,energy_da,energy_rt,load,gen_biomas,gen_coal,gen_gas,gen_gascc,gen_hydro,gen_nuclear,gen_other,gen_solar,gen_wsl,gen_wind
time_tomerge,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00+00:00,5.26,5.00,6.04,1.08,20.533333,13.808125,"37,258.99",12.225,1593.075,145.750,2983.325,46.700,973.675,0.575,0.00,NaN,3578.450
2019-01-01 02:00:00+00:00,3.98,4.37,5.90,1.05,19.813333,14.866875,"37,300.19",12.300,1571.375,147.375,3063.625,46.725,973.700,0.275,0.00,NaN,3485.250
2019-01-01 03:00:00+00:00,3.50,4.44,5.01,0.93,19.593333,15.502500,"37,423.54",12.250,1615.050,149.175,3155.725,46.825,973.550,0.850,0.00,NaN,3381.975
2019-01-01 04:00:00+00:00,3.47,5.00,5.63,0.93,19.241667,16.103125,"37,895.21",12.100,1639.525,196.925,3229.850,46.700,973.400,0.400,0.00,NaN,3343.475
2019-01-01 05:00:00+00:00,8.00,7.00,7.02,0.94,21.266667,16.032500,"38,825.09",12.050,1708.550,277.750,3462.950,53.500,973.550,0.875,0.00,NaN,3158.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00+00:00,7.00,2.29,1.95,1.00,16.875714,17.927000,"40,881.88",2.725,1257.600,225.675,2501.300,1.175,1277.500,19.675,0.00,-12.775,5041.775
2022-12-31 20:00:00+00:00,3.50,1.50,1.65,1.00,13.160000,9.171500,"39,635.70",2.700,1177.875,219.925,2119.450,1.100,1277.700,12.050,0.05,-25.325,5334.850
2022-12-31 21:00:00+00:00,4.51,1.50,1.50,0.99,11.960000,7.474500,"38,605.05",2.725,1174.925,218.600,1807.950,1.150,1277.675,6.175,0.00,-22.725,5489.775


In [213]:
total = pd.concat([old, combined])


In [ ]:
total.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs/ERCOT.csv")